In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [9]:
indep.shape

(400, 4)

In [10]:
indep

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [13]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
param_grid ={'alpha':[1.0], 'force_alpha':[True,False], 'fit_prior':[True,False], 'class_prior':[None], 'norm':[True,False] }
grid = GridSearchCV(ComplementNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [1.0], 'class_prior': [None],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False], 'norm': [True, False]},
             scoring='f1_weighted', verbose=3)

In [28]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(X_test)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(Y_test, Y_pred)

#print(clf_report)

In [30]:
from sklearn.metrics import f1_score

f1_macro=f1_score(Y_test,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 1.0, 'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'norm': True}: 0.7568518518518519


In [32]:
print("The confusion Matrix:\n",cm)
print("The report:\n",clf_report)

The confusion Matrix:
 [[77  2]
 [24 17]]
The report:
               precision    recall  f1-score   support

           0       0.76      0.97      0.86        79
           1       0.89      0.41      0.57        41

    accuracy                           0.78       120
   macro avg       0.83      0.69      0.71       120
weighted avg       0.81      0.78      0.76       120



In [34]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.6820006174745292

In [36]:
table=pd.DataFrame.from_dict(result)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011204,0.003778,4.705279,2.365936,1.0,None,True,True,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.795918,0.763066,0.618196,0.771493,0.693083,0.728351,0.064822,1
1,0.012927,0.005330,0.012203,0.004293,1.0,None,True,True,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,5
2,0.012043,0.002158,0.012100,0.002835,1.0,None,True,False,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.795918,0.763066,0.618196,0.771493,0.693083,0.728351,0.064822,1
3,0.012873,0.003988,0.017363,0.006233,1.0,None,True,False,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,5
4,0.014366,0.004446,0.015198,0.001664,1.0,None,False,True,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.795918,0.763066,0.618196,0.771493,0.693083,0.728351,0.064822,1
5,0.010573,0.003149,0.018689,0.003875,1.0,None,False,True,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,5
6,0.016564,0.006826,0.024600,0.011869,1.0,None,False,False,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.795918,0.763066,0.618196,0.771493,0.693083,0.728351,0.064822,1
7,0.014921,0.002766,0.015460,0.002308,1.0,None,False,False,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.607143,0.733523,0.638285,0.630424,0.714286,0.664732,0.049761,5
